In [ ]:
# Install necessary libraries if not already installed
!pip install groq together openai
#never mind this  is the best
import json
from together import Together
from openai import OpenAI


# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    if conversation_history:
        formatted_history = " ".join(conversation_history)
    else:
        formatted_history = "none"
    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives STT (speech-to-text) inputs from customers placing coffee orders. Your task is to process these inputs, respond in Korean, and generate a JSON output for backend processing.

    **Menu Items**:
    - Hot Drinks: 허브티 (always hot)
    - Iced Only Drinks: 토마토주스, 키위주스, 망고스무디, 딸기스무디, 레몬에이드, 복숭아아이스티 (always iced)
    - Hot/Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
    - Specialty: 초콜릿라떼 (hot or iced)

    **Default Values**:
    - Use default size "미디움" and temperature "핫" if unspecified.
    - Do not override explicitly given size or temperature.

    **Response Requirements**:
    1. **Natural Language Response**: Confirm each item in Korean, e.g., "what action has been done based on latest output.", followed by a full summary of all ordered items, starting with "지금까지 주문하신 내용은 다음과 같습니다:".
    2. **JSON Output**: Only include items from the latest input in the structured JSON format below:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons],
                  "extra_shots": [Number of extra shots]
              }}
          ]
      }}
    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Key Scenarios**:

    - New Order: Confirm with a natural response and JSON output for each new drink.
    - Modification: Confirm changes and modify JSON.
    - Summary Request: Provide a summary without a JSON output.
    - Unavailable Items: Recommend a similar item.
    - Order Completion: Confirm completion and provide a summary.

    Current Conversation History: {formatted_history}

    Response: """

    print(" **Current Conversation History**:", formatted_history)

    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct-Lite",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    # Try to split the response based on "JSON Output:" to extract the natural language and JSON parts
    if "JSON Output:" in response:
        try:
            natural_language_response, json_output = response.split("JSON Output:", 1)
            json_data = json.loads(json_output.strip())
            return natural_language_response.strip(), json_data
        except json.JSONDecodeError as e:
            # If there's an error parsing JSON, return the natural response and an empty dictionary
            print("Error decoding JSON:", e)
            return response.strip(), {}
    else:
        # If "JSON Output:" is missing, print a message and return the whole response for debugging
        print("Warning: 'JSON Output:' not found in the model response.")
        return response.strip(), {}


# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")
input_counter = 1

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        conversation_history.clear()
        summary_history.clear()
        continue
    # Append the new input to conversation history with labeled format
    conversation_history.append(f"Customer's {input_counter} Input: {user_input}")

    # Generate the prompt based on the conversation history and new user input
    prompt = generate_prompt(conversation_history, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)


    # Update the cumulative summary with only the latest relevant order information
    summary_line = response.split("\n", 1)[1] if "\n" in response else response
    cumulative_summary = summary_line

    # Print the final natural language response for the customer
    print("Kiosk:", natural_language_response)

    input_counter += 1

    # Special handling for order summary requests
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Provide cumulative order summary as a natural response without JSON output
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(summary_history))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear conversation history and summary history for a reset
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        conversation_history.clear()
        summary_history.clear()
        input_counter = 1

# End of the session
print("Thank you for using the coffee kiosk!")


Welcome to the virtual coffee kiosk! What would you like to order?
Customer: Customer’s 1 Input: 아이스 카페 라떼 라지 두 잔과 바닐라 시럽 추가한 카페 라떼 한 잔 주세요.
 **Current Conversation History**: Customer's 1 Input: Customer’s 1 Input: 아이스 카페 라떼 라지 두 잔과 바닐라 시럽 추가한 카페 라떼 한 잔 주세요.
Kiosk: 지금까지 주문하신 내용은 다음과 같습니다: 아이스 카페 라떼 2잔과 바닐라 시럽 추가한 카페 라떼 1잔. 

{
  "action": "create_order",
  "order_items": [
    {
      "drink": "카페 라떼",
      "size": "라지",
      "temperature": "아이스",
      "quantity": 2,
      "add_ons": ["바닐라 시럽"],
      "extra_shots": 0
    },
    {
      "drink": "카페 라떼",
      "size": "미디움",
      "temperature": "아이스",
      "quantity": 1,
      "add_ons": ["바닐라 시럽"],
      "extra_shots": 0
    }
  ]
}


KeyboardInterrupt: Interrupted by user